## Neural Networks with Scikit-Learn: Regression vs Classification

> For more info [here](https://coderzcolumn.com/tutorials/machine-learning/scikit-learn-sklearn-neural-network)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn

import itertools
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=2)
%matplotlib inline

-----

## MLPClassifier

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X_digits, Y_digits = digits.data, digits.target
print('Dataset Sizes : ', X_digits.shape, Y_digits.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_digits, Y_digits, train_size=0.80, test_size=0.20, stratify=Y_digits, random_state=123)
print('Train/Test Sizes : ', X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_classifier  = MLPClassifier(random_state=123)
mlp_classifier.fit(X_train, Y_train)

In [ ]:
Y_preds = mlp_classifier.predict(X_test)

print(Y_preds[:15])
print(Y_test[:15])

print('Test Accuracy : %.3f'%mlp_classifier.score(X_test, Y_test)) ## Score method also evaluates accuracy for classification models.
print('Training Accuracy : %.3f'%mlp_classifier.score(X_train, Y_train))

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(Y_test, Y_preds):
    conf_mat = confusion_matrix(Y_test, Y_preds)
    print(conf_mat)
    fig = plt.figure(figsize=(6,6))
    plt.matshow(conf_mat, cmap=plt.cm.Blues, fignum=1)
    plt.yticks(range(10), range(10))
    plt.xticks(range(10), range(10))
    plt.colorbar();
    for i in range(10):
        for j in range(10):
            plt.text(i-0.2,j+0.1, str(conf_mat[j, i]), color='tab:red')

In [ ]:
plot_confusion_matrix(Y_test, mlp_classifier.predict(X_test))

### Important Attributes of MLPClassifier

Below is a list of important attributes available with an MLPClassifier which can provide meaningful insights once the model is trained.

    loss_ - Loss after the training process has completed.
    coefs_ - An array of length n_layers-1 where each element represents weights associated with layer i.
    intercepts_ - An array of length n_layers-1 where each element represents intercept associated with layer i's perceptrons.
    n_iter_ - The number of iterations for which estimator ran.
    out_activation_ - Name of output layer activation function.



In [ ]:
print("Loss : ", mlp_classifier.loss_)

In [ ]:
print("Number of Coefs : ", len(mlp_classifier.coefs_))

[weights.shape for weights in mlp_classifier.coefs_]

In [ ]:
print("Number of Intercepts : ", len(mlp_classifier.intercepts_))

[intercept.shape for intercept in mlp_classifier.intercepts_]

In [ ]:
print("Number of Iterations for Which Estimator Ran : ", mlp_classifier.n_iter_)

In [ ]:
print("Name of Output Layer Activation Function : ", mlp_classifier.out_activation_)

### GridSearchCV

It's a wrapper class provided by sklearn which loops through all parameters provided as params_grid parameter with a number of cross-validation folds provided as cv parameter, evaluates model performance on all combinations and stores all results in cv_results_ attribute. It also stores model which performs best in all cross-validation folds in best_estimator_ attribute and best score in best_score_ attribute.

In [ ]:
from sklearn.model_selection import GridSearchCV
'''
params = {'activation': ['relu', 'tanh', 'logistic', 'identity'],
          'hidden_layer_sizes': [(100,), (50,100,), (50,75,100,)],
          'solver': ['adam', 'sgd', 'lbfgs'],
          'learning_rate' : ['constant', 'adaptive', 'invscaling']
         }
'''
params = {'activation': ['relu', 'tanh'],
          'hidden_layer_sizes': [(100,), (50,100,), (50,75,100,)],
          'solver': ['adam', 'sgd'],
          'learning_rate' : ['constant']
         }

mlp_classif_grid = GridSearchCV(MLPClassifier(random_state=123), param_grid=params, n_jobs=-1, cv=2, verbose=5)
mlp_classif_grid.fit(X_train,Y_train)

print('Train Accuracy : %.3f'%mlp_classif_grid.best_estimator_.score(X_train, Y_train))
print('Test Accuracy : %.3f'%mlp_classif_grid.best_estimator_.score(X_test, Y_test))
print('Best Accuracy Through Grid Search : %.3f'%mlp_classif_grid.best_score_)
print('Best Parameters : ',mlp_classif_grid.best_params_)

In [ ]:
plot_confusion_matrix(Y_test, mlp_classif_grid.best_estimator_.predict(X_test))

--------

## MLPRegressor


### Boston housing prices dataset : Ethical concerns

The Boston housing prices dataset has an ethical problem: as
investigated [here](https://medium.com/@docintangible/racist-data-destruction-113e3eff54a8).



In [ ]:
#from sklearn.datasets import load_boston

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In [ ]:
    URL = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(URL, sep="\s+", skiprows=22, header=None)
    X_boston = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    Y_boston  = raw_df.values[1::2, 2]
    print('Dataset Sizes : ', X_boston.shape, Y_boston.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_boston, Y_boston, train_size=0.80, test_size=0.20, random_state=123)
print('Train/Test Sizes : ', X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_regressor  = MLPRegressor(random_state=123)
mlp_regressor.fit(X_train, Y_train)

In [ ]:
Y_preds = mlp_regressor.predict(X_test)

print(Y_preds[:10])
print(Y_test[:10])

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, Y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, Y_train))

### Important Attributes of MLPRegressor

In [ ]:
print("Loss : ", mlp_regressor.loss_)

In [ ]:
print("Number of Coefs : ", len(mlp_regressor.coefs_))

[weights.shape for weights in mlp_regressor.coefs_]

In [ ]:
print("Number of Intercepts : ", len(mlp_regressor.intercepts_))

[intercept.shape for intercept in mlp_regressor.intercepts_]

In [ ]:
print("Number of Iterations for Which Estimator Ran : ", mlp_regressor.n_iter_)

In [ ]:
print("Name of Output Layer Activation Function : ", mlp_regressor.out_activation_)

### GridSearchCV



In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Define the parameter grid
params = {
    'activation': ['relu', 'tanh', 'logistic', 'identity'],
    'hidden_layer_sizes': [(100,), (50,100,), (50,75,100,)],
    'solver': ['adam'],
    'learning_rate': ['constant']
}

# Create MLPRegressor and GridSearchCV objects
mlp_regressor_grid = GridSearchCV(MLPRegressor(random_state=123),
                                  param_grid=params,
                                  n_jobs=-1,
                                  cv=2,
                                  verbose=5,
                                  error_score='raise')  # Set error_score to raise exceptions

# Fit the model with scaled data
mlp_regressor_grid.fit(X_train_scaled, Y_train)

# Print results
print('Train Score (R^2): %.3f' % mlp_regressor_grid.best_estimator_.score(X_train_scaled, Y_train))
print('Best Parameters:', mlp_regressor_grid.best_params_)


In [ ]:
# Predict on test set
predictions = mlp_regressor_grid.best_estimator_.predict(X_test)

# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(Y_test, predictions, c='blue')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values')
plt.show()


## Ref

[https://coderzcolumn.com/tutorials/machine-learning/scikit-learn-sklearn-neural-network](https://coderzcolumn.com/tutorials/machine-learning/scikit-learn-sklearn-neural-network)